## Inhalt des Skripts
# Finrep: Erhebung 51

Ziele des Skripts ist pro START Konzept, vorerst in erster Linie ISIS Konzepte über ein Mapping von Konzept zu Datapoint aus dem DPM Metadaten zu extrahieren. 
OSIRIS Konzepte sind noch offen. Hier warten wir noch bis sämtliche Konzepte im Datenkatalog sind und fragen dann nach einen Abzug

# Setup

In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np
import chardet


## START Konzepte

In [2]:
#File von Dominik/Christina zur Initialbefüllung des Datenkatalogs
# Relativer Pfad zur Datei
excel_file_path = "Files/ISIS-Erhebungsstammdaten1.xlsx"  

# Spezifisches Sheet einlesen
df_konzepte = pd.read_excel(excel_file_path, sheet_name="Gemeldete Konzepte")  

# Zeige die ersten Zeilen der Tabelle
print(df_konzepte.columns)

df_konzepte = df_konzepte[['Code' , 'Kurzbezeichnung (englisch)','Erhebungsteile']]
print(df_konzepte.shape)
df_konzepte.head(1)

Index(['Code', 'Smart Cube Konzept', 'Pflichtkonzept', 'Dimensionskombination',
       'Dimensionen', 'Konzepttyp', 'OBServ-Schlüsselgruppe',
       'SCS-Einschränkung', 'Anubis-Rechenregel', 'Aggregationstyp',
       'Kurzbezeichnung', 'Kurzbezeichnung (englisch)', 'Bezeichnung',
       'Bezeichnung (englisch)', 'Beschreibung', 'Gültig von', 'Gültig bis',
       'MDI relevant', 'MDI Modellierungstyp', 'Erhebungsteile'],
      dtype='object')
(165342, 3)


,Code,Kurzbezeichnung (englisch),Erhebungsteile
0,IS01_FMA,NaN,NaN


In [3]:
#check for duplicates
has_duplicates = df_konzepte['Code'].duplicated().any()

print(has_duplicates)

df_konzepte = df_konzepte.rename(columns={"Code": "code"})

df_konzepte.head(1)


False


,code,Kurzbezeichnung (englisch),Erhebungsteile
0,IS01_FMA,NaN,NaN


In [4]:
#filtere auf Finrep-Erhebungen mit "51-F"
df_konzepte = df_konzepte[df_konzepte["Erhebungsteile"].str.contains("51-F", na=False)]
print(df_konzepte.shape)


(13275, 3)


In [5]:
#Stichprobe-Test
df_konzepte[df_konzepte["code"] == "ISFIN0000001"]

,code,Kurzbezeichnung (englisch),Erhebungsteile
91697,ISFIN0000001,Cash and cash bal at CB,"51-F01.01, 56-F01.01, FINAI-0101, FINAU-0101, ..."


# Mapping START Konzepte zu DPM Datapoint
## Verwendung von ITS Base Data

In [1]:

"""Importieren der benötigeten Packages"""
#pip install requests
#pip install requests-kerberos
#pip install pandas
#pip install beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests_kerberos import HTTPKerberosAuth, OPTIONAL
import re
import os  
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, DateType
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
from pyspark_llap import HiveWarehouseSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os
 
################# USER INPUT ########################

hue_databank_name = "its_analyse_test"
table_name = "its_base_data"

#####################################################

hive_table = f"{hue_databank_name}.{table_name}"

# Spark Session with necessary configurations for Hive transactions
spark = SparkSession.builder\
    .appName("hwc-app")\
    .config("spark.security.credentials.hiveserver2.enabled","false")\
    .config("spark.datasource.hive.warehouse.read.via.llap","false")\
    .config("spark.datasource.hive.warehouse.read.jdbc.mode", "client")\
    .config("spark.sql.hive.hiveserver2.jdbc.url","jdbc:hive2://anucdp-mgmt-01.w.oenb.co.at:2181,anucdp-mgmt-02.w.oenb.co.at:2181,anucdp-mgmt-03.w.oenb.co.at:2181/;serviceDiscoveryMode=zooKeeper;zooKeeperNamespace=hiveserver2;trustStoreType=jks;ssl=true")\
    .config("spark.yarn.historyServer.address","https://anucdp-mgmt-02.w.oenb.co.at")\
    .config("spark.sql.hive.hiveserver2.jdbc.url.principal","hive/_HOST@AD.OENB.CO.AT")\
    .config("spark.hadoop.yarn.resourcemanager.principal","hive")\
    .config("spark.kryo.registrator","com.qubole.spark.hiveacid.util.HiveAcidKyroRegistrator")\
    .config("spark.sql.extensions","com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension")\
    .config("spark.jars", "/runtime-addons/spark332-7190-1202-b75-ht9wmb/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar")\
    .getOrCreate()

# Initialize HiveWarehouseSession
hwc = HiveWarehouseSession.session(spark).build()

# Explicitly set the database using SQL
hwc.setDatabase(f'{hue_databank_name}')
hwc.sql(f"USE {hue_databank_name}").show()

Setting spark.hadoop.yarn.resourcemanager.principal to gaubet


++
||
++
++



In [2]:
# Abfrage, um alle Zeilen mit dem spezifischen Wert in der Spalte "module_gueltig_bis" zu erhalten
filtered_rows = hwc.sql("""
    SELECT * 
    FROM its_analyse_test.its_base_data
    WHERE module_gueltig_bis = '3099-12-31 00:00:00'
""")
#filtered_rows.show()
data = filtered_rows.collect()  # Liefert eine Liste von Row-Objekten zurück


# Umwandeln in Pandas DataFrame
pandasDF = pd.DataFrame([row.asDict() for row in data], columns=filtered_rows.columns)
print(pandasDF.shape)
pandasDF.head(1)

(87341, 18)


,datapoint,konzept_code,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
0,10678,ISFIN6080000,FINREP_3.2.1-DP,F 16.04,Gains and losses on financial assets and liabi...,FINREP9_DP,2022-12-31,3099-12-31 00:00:00,F 16.04,None,None,None,None,None,None,None,None,8da304c9629877b7955dca498a7dca5c


In [8]:
#analyse 
pandasDF["taxonomy_code"]
distinct_values_list = pandasDF["taxonomy_code"].unique().tolist()
print(distinct_values_list)

['FINREP_3.2.1-DP', 'FINREP_3.2.1', 'COREP_3.2', 'RES_3.2.1', 'AE_3.2', 'ResRep_2024', 'FP_3.2.1', 'COVID19_3.2.1', 'REM_3.2.2', 'ESG_3.3', 'SBP_3.3.1', 'GSII_3.2', 'IPU_3.3']


In [9]:
#Strichprobentest
filtered_df = pandasDF[pandasDF["konzept_code"] == "ISFIN0000001"]
filtered_df

,datapoint,konzept_code,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
37140,112718,ISFIN0000001,FINREP_3.2.1,F 01.01,Balance Sheet Statement [Statement of Financia...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 01.01,Balance Sheet Statement [Statement of Financia...,None,0010,Carrying amount,0020,Cash on hand,None,None,f86f057e7c8f2bf77c973f23b8aa5043
37564,112718,ISFIN0000001,FINREP_3.2.1-DP,F 01.01_dp,Balance Sheet Statement [Statement of Financia...,FINREP9_DP,2022-12-31,3099-12-31 00:00:00,F 01.01_dp,None,None,None,None,None,None,None,None,d072b79c7653abcd5131bebcf79a8d93
82172,112718,ISFIN0000001,FINREP_3.2.1-DP,F 01.01,Balance Sheet Statement [Statement of Financia...,FINREP9_DP,2022-12-31,3099-12-31 00:00:00,F 01.01,None,None,None,None,None,None,None,None,12b735b5a3b17a4d877c2ca14afa846f


In [10]:
df_its_filtered = pandasDF[pandasDF["taxonomy_code"] == "FINREP_3.2.1"]
print(df_its_filtered.shape)
df_its_filtered.head(1)

(13860, 18)


,datapoint,konzept_code,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
1,11133,ISFIN0001590,FINREP_3.2.1,F 14.00,Fair value hierarchy: financial instruments at...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 14.00,Fair value hierarchy: financial instruments at...,None,0060,Level 1,0040,Debt securities,None,None,8a8f74d190a039b2267f136dc0c14533


In [11]:
anzahl_duplikate = df_its_filtered.duplicated(subset=['datapoint']).sum()
print(anzahl_duplikate)
duplizierte_zeilen = df_its_filtered[df_its_filtered.duplicated(subset=['datapoint'], keep=False)]
duplizierte_zeilen = duplizierte_zeilen.sort_values(by='datapoint', ascending=False)
#duplizierte_zeilen

1227


In [12]:
#Ich möchte feststellen, ob es pro konzept_code unterschiedliche datapoints gibt.
unique_counts = df_its_filtered.groupby('konzept_code')['datapoint'].nunique()

# unique_counts ist nun eine Series, in der der Index die konzept_codes sind
# und die Werte angeben, wie viele unterschiedliche datapoints es pro konzept_code gibt.

# Um alle konzept_codes aufzulisten, die mehr als einen unterschiedlichen datapoint haben:
konzepte_mehrere_datapoints = unique_counts[unique_counts > 1]

print(konzepte_mehrere_datapoints)


Series([], Name: datapoint, dtype: int64)


## Mapping Finrep-Konzepte zu datapoint

In [13]:
df_its_filtered = df_its_filtered.rename(columns={"konzept_code": "code"})
df_its_filtered.head(1)

,datapoint,code,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
1,11133,ISFIN0001590,FINREP_3.2.1,F 14.00,Fair value hierarchy: financial instruments at...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 14.00,Fair value hierarchy: financial instruments at...,None,0060,Level 1,0040,Debt securities,None,None,8a8f74d190a039b2267f136dc0c14533


In [18]:
merged_df = pd.merge(df_konzepte, df_its_filtered, on="code", how="left", validate="one_to_many")
print(merged_df.shape)
merged_df.head(1)

(13683, 20)


,code,Kurzbezeichnung (englisch),Erhebungsteile,datapoint,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
0,ISFIN0003864,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67726.0,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0010,Selected financial assets recognised in the re...,None,None,b8791e6e8b981f89a1483552d43a40cb


In [16]:
#df_mapping_g["taxonomy_code"]
#distinct_values_list = merged_df["taxonomy_code"].unique().tolist()
#print(distinct_values_list)


['FINREP_3.2.1', nan]


In [ ]:
#wir mergen über einen Left - join die Mapping daten zu den Konzepten hinzu. mit validate: one_to_many
#merged_df = pd.merge(df_konzepte, df_mapping_g, on="code", how="left", validate="one_to_many")

#print(merged_df.shape)
#print(merged_df.dtypes)
#merged_df.head(5)
#merged_df.to_csv("merged_df.csv", index=False)

In [19]:
#wir droppen alle Zeilen ohne datapoint, wir wandeln datapoint in int um, der wird über merge zu float
df_cleaned = merged_df.dropna(subset=['datapoint'])
df_cleaned['datapoint'] = df_cleaned['datapoint'].astype('int')
print(df_cleaned.dtypes)
print(df_cleaned.shape)
df_cleaned.head()

code                                  object
Kurzbezeichnung (englisch)            object
Erhebungsteile                        object
datapoint                              int64
taxonomy_code                         object
template_id                           object
template_label                        object
module_id                             object
module_gueltig_von            datetime64[ns]
module_gueltig_bis                    object
table_id                              object
table_name                            object
criteria                              object
x_axis_rc_code                        object
x_axis_name                           object
y_axis_rc_code                        object
y_axis_name                           object
z_axis_rc_code                        object
z_axis_name                           object
idt_hash                              object
dtype: object
(11541, 20)


/tmp/ipykernel_279/3100463229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['datapoint'] = df_cleaned['datapoint'].astype('int')


,code,Kurzbezeichnung (englisch),Erhebungsteile,datapoint,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
0,ISFIN0003864,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67726,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0010,Selected financial assets recognised in the re...,None,None,b8791e6e8b981f89a1483552d43a40cb
1,ISFIN0003865,Sel fin As recog in bal,"51-F30.00, FINKF-3000",109614,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0021,of which: non-performing,None,None,aa6a4ce2ba1b5fb3f6290a63fca48551
2,ISFIN0003866,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67722,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0030,Derivatives,None,None,2ccb930085253428ec7caa76fe5f8dcd
3,ISFIN0003867,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67735,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0040,Equity instruments,None,None,55244579bf128513555e4852fb31a0d7
4,ISFIN0003868,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67715,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 30.02,Breakdown of interests in unconsolidated struc...,None,0010,Securitisation Special Purpose Entities,0050,Debt securities,None,None,29caac5245c732d9541e90ecac9b3d5c


## Import DPM data

In [2]:
#K:\HST-IT-Systeme\TP_2\Umsetzung\XBRL\EBA\DPM_3.5.0_Release_09_07_2024\dpm_databse_3.5_dpm_1.0.zip 
csv_file_path = "Files/qDPM_DataPointCategorisations.csv"  

#with open("Files/qDPM_DataPointCategorisations.csv", 'rb') as file:
#    result = chardet.detect(file.read())
#    print(result)



In [3]:
df_dpm = pd.read_csv(csv_file_path, delimiter=';', encoding='Windows-1252', header=None, names= ["DataPointVID", "DataPointID", "DimensionLabel", "MemberName"])
print(df_dpm.shape)

distinct_values_list_dpm = df_dpm["MemberName"].unique().tolist()
#print(distinct_values_list_dpm)
print(len(distinct_values_list_dpm))

df_dpm.head()
#df_dpm.size


(1438078, 4)
4828


,DataPointVID,DataPointID,DimensionLabel,MemberName
0,10002,10002,Type of allowance,All allowances
1,10002,10002,Accounting portfolio,Accounting portfolios for financial assets sub...
2,10002,10002,Metric,Amount of accumulated impairment [mi]
3,10002,10002,Base,Assets
4,10002,10002,Main category,"Equity instruments, debt securities, loans and..."


## Merge Konzept über DataPointVID

In [22]:
#rename DataPointVID zu datapoint für merge
df_dpm = df_dpm.rename(columns={"DataPointVID": "datapoint"})

In [23]:
merged_df_final = pd.merge(merged_df, df_dpm, on="datapoint", how="left")

In [24]:
print(merged_df_final.shape)
distinct_values_list_dpm2 = merged_df_final["MemberName"].unique().tolist()
print(len(distinct_values_list_dpm2))
distinct_values_list_dpm3 = merged_df_final["code"].unique().tolist()
print(len(distinct_values_list_dpm3))
merged_df_final.head(1)

(78987, 23)
602
13275


,code,Kurzbezeichnung (englisch),Erhebungsteile,datapoint,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,...,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash,DataPointID,DimensionLabel,MemberName
0,ISFIN0003864,Sel fin As recog in bal,"51-F30.00, FINKF-3000",67726.0,FINREP_3.2.1,F 30.02,Breakdown of interests in unconsolidated struc...,FINREP9,2022-12-31,3099-12-31 00:00:00,...,0010,Securitisation Special Purpose Entities,0010,Selected financial assets recognised in the re...,None,None,b8791e6e8b981f89a1483552d43a40cb,67726.0,Accounting portfolio,Accounting portfolios for financial assets oth...


In [36]:
merged_df_final.to_csv('output_20250108.csv', index=False)


# Merge with reference 

In [31]:

#distinct_values = merged_df_final['template_id'].unique()
#print(distinct_values)

df_1_1 = merged_df_final[merged_df_final['template_id'] == 'F 01.01'] 
df_1_1.head(1)

,code,Kurzbezeichnung (englisch),Erhebungsteile,datapoint,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,...,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash,DataPointID,DimensionLabel,MemberName
2440,ISFIN0006476,NaN,"51-F01.01, FINAI-0101, FINID-0101, FININ-0101,...",152251.0,FINREP_3.2.1,F 01.01,Balance Sheet Statement [Statement of Financia...,FINREP9,2022-12-31,3099-12-31 00:00:00,...,0010,Carrying amount,0181,Financial assets at amortised cost,None,None,e7d5533c001d73dda662d1f176908cbc,152251.0,Accounting portfolio,Financial assets at amortised cost


In [41]:
#df_1_1[df_1_1['y_axis_rc_code'] == "0370"].head()

In [48]:
df_reference = pd.read_csv('output_reference_test.csv')
df_reference.rename(columns={'Description': 'y_axis_rc_code'}, inplace=True)
df_reference.head(10)


,y_axis_rc_code,Code,Reference,Worksheet
0,NaN,NaN,NaN,1.1
1,1.1 Assets,NaN,NaN,1.1
2,NaN,NaN,NaN,1.1
3,NaN,NaN,References,1.1
4,NaN,NaN,NaN,1.1
5,NaN,NaN,NaN,1.1
6,0010,"Cash, cash balances at central banks and other...",IAS 1.54 (i),1.1
7,0020,Cash on hand,Annex V.Part 2.1,1.1
8,0030,Cash balances at central banks,Annex V.Part 2.2,1.1
9,0040,Other demand deposits,Annex V.Part 2.3,1.1


In [49]:
result = pd.merge(df_1_1, df_reference, how='left', on='y_axis_rc_code')
result.head(1)

In [70]:
pd.set_option('display.max_colwidth', None)

result['match'] = result.apply(lambda row: True if row['MemberName'] == row['Code'] else False, axis=1)
#result[result['match']].sort_values(by='y_axis_rc_code', ascending=True)
result[result['y_axis_rc_code']=='0010'][['MemberName', 'Code', 'match']]

,MemberName,Code,match
60,Cash and cash balances at central banks and other demand deposits,"Cash, cash balances at central banks and other demand deposits",False
61,Carrying amount [mi],"Cash, cash balances at central banks and other demand deposits",False
62,Assets,"Cash, cash balances at central banks and other demand deposits",False
63,"Cash on hand, Loans and advances. On demand [call] and short notice [current account]","Cash, cash balances at central banks and other demand deposits",False


# Similarity Scores


In [82]:
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Lade das vortrainierte Modell von spaCy
nlp = spacy.load("en_core_web_md")

In [84]:
# Fuzzy Matching
def calculate_fuzzy_similarity(row):
    return fuzz.ratio(row['MemberName'], row['Code']) / 100  # Normalisiere auf 0-1

result['Fuzzy_Similarity'] = result.apply(calculate_fuzzy_similarity, axis=1)

# TF-IDF Similarity
def calculate_tfidf_similarity(row):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([row['MemberName'], row['Code']])
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1]).flatten()[0]

result['TFIDF_Similarity'] = result.apply(calculate_tfidf_similarity, axis=1)



# Semantische Ähnlichkeit (spaCy)
def calculate_semantic_similarity(row):
    doc1 = nlp(row['MemberName'])
    doc2 = nlp(row['Code'])
    return doc1.similarity(doc2)

result['Semantic_Similarity'] = result.apply(calculate_semantic_similarity, axis=1)

result[result['y_axis_rc_code']=='0010'][['MemberName', 'Code', 'match', 'Fuzzy_Similarity', 'TFIDF_Similarity', 'Semantic_Similarity']]

,MemberName,Code,match,Fuzzy_Similarity,TFIDF_Similarity,Semantic_Similarity
60,Cash and cash balances at central banks and other demand deposits,"Cash, cash balances at central banks and other demand deposits",False,0.96,0.968963,0.994944
61,Carrying amount [mi],"Cash, cash balances at central banks and other demand deposits",False,0.24,0.000000,0.567244
62,Assets,"Cash, cash balances at central banks and other demand deposits",False,0.15,0.000000,0.702872
63,"Cash on hand, Loans and advances. On demand [call] and short notice [current account]","Cash, cash balances at central banks and other demand deposits",False,0.39,0.217130,0.860652
